In [1]:
import pandas as pd
import numpy as np
from scipy import stats
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
import json
from IPython.display import display
from subprocess import check_output
import sys
import traceback
from tqdm.notebook import tqdm, trange
import plotly.graph_objects as go
import plotly.express as px
pd.options.mode.chained_assignment = None
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
techs = {
    "tech1": "1q_1o",
    "tech2": "k_per_terra",
    "tech3": "4pip",
    "tech4": "7vp",
    "tech5": "1o_1pw",
    "tech6": "1k_1c",
    "tech7": "3vp_per_gaia_place",
    "tech8": "4c",
    "tech9": "4pw",
    # idk wtf this is --> "tech-ship0"
}

adv_techs = {
    "advtech1": "3vp_per_fed_pass",
    "advtech2": "2vp_per_tech_bump",
    "advtech3": "1qic_5c_action",
    "advtech4": "2vp_per_mine",
    "advtech5": "3vp_per_rl_pass",
    "advtech6": "1o_per_sector",
    "advtech7": "1vp_per_terra_pass",
    "advtech8": "2vp_per_gaia",
    "advtech9": "4vp_per_ts",
    "advtech10": "2vp_per_sector",
    "advtech11": "3o_action",
    "advtech12": "5vp_per_fed",
    "advtech13": "3k_action",
    "advtech14": "3vp_per_mine_place",
    "advtech15": "3vp_per_ts_place",
}

feds = {
    "fed1": "12vp",
    "fed2": "qic",
    "fed3": "2pw",
    "fed4": "2o",
    "fed5": "6c",
    "fed6": "2k",
    "gleens": "gleens"
}

round_scorings = {
    "score1": "2vp_per_terra",
    "score2": "2vp_per_research_bump",
    "score3": "2vp_per_mine_place",
    "score4": "5vp_per_fed_place",
    "score5": "4vp_per_ts_place",
    "score6": "4vp_per_gaia_place",
    "score7": "5vp_per_3pip_place",
    "score8": "3vp_per_ts_place",
    "score9": "3vp_per_gaia_place",
    "score10": "5vp_per_3pip_place",
}

boosters = {

    "booster1": "1k_1o",
    "booster2": "2pwt_1o",
    "booster3": "1qic_2c",
    "booster4": "2c_terra",
    "booster5": "2pw_nav",
    "booster6": "1o_1vp_per_mine",
    "booster7": "1o_2vp_per_ts",
    "booster8": "1k_3vp_per_rl",
    "booster9": "4pw_4vp_per_3pip",
    "booster10": "4c_1vp_per_gaia",
}

bad_buildings = {"colony", 
                 "colonyShip", 
                 "tradeShip", 
                 "constructionShip", 
                 "researchShip",
                 "scout",
                 "frigate",
                 "battleShip",
                 "customsPost",
                 "tradePost"
                }

factions = {
    "terrans",
    "lantids",
    "hadsch-hallas",
    "ivits",
    "baltaks",
    "geodens",
    "xenos",
    "gleens",
    "ambas",
    "taklons",
    "bescods",
    "firaks",
    "itars",
    "nevlas"
}

In [8]:
dat_2 = pd.read_pickle("./data/two_players_data")
dat_3 = pd.read_pickle("./data/three_players_data")
dat_4 = pd.read_pickle("./data/four_players_data")

In [9]:
dat_4['pos_1_dropped'].describe()

count      5681
unique        2
top       False
freq       5665
Name: pos_1_dropped, dtype: object

In [12]:
def plot_r1_structs_remote(num_players, dat_x):
    # do cleaning
    dat_clean = dat_x.copy()
    for pos in range(num_players):
        dat_clean = dat_clean[dat_clean['pos_' + str(pos + 1) + '_dropped'] == False]
        dat_clean['pos_' + str(pos + 1) + '_score'] += abs(dat_clean['pos_' + str(pos + 1) + '_score_bid'])
        dat_clean = dat_clean[dat_clean['pos_' + str(pos + 1) + '_score'] >= 100]

    for faction in factions:
        factions_pos = []
        for pos in range(num_players):
            dat = dat_clean.copy()

            # select faction
            dat = dat[dat["pos_" + str(pos + 1) + "_faction"] == faction]

            # remove dropped players
            # dat = dat[dat['pos_' + str(pos + 1) + '_dropped'] == False]

            # bid part of score
            # dat['pos_' + str(pos + 1) + '_score'] += -1 * dat['pos_' + str(pos + 1) + '_score_bid']

            # labels
            prefix = "pos_" + str(pos + 1) + '_buildings_r_1'
            structs_class = 'r1_structs_class_' + faction
            structs_exact = 'r1_structs_exact_' + faction
            over_all_score = 'ave_score_' + faction
            over_all_elo = 'ave_elo_' + faction

            # get scores and elos
            dat[over_all_score] = dat['pos_' + str(pos + 1) + '_score']
            dat[over_all_elo] = dat['pos_' + str(pos + 1) + '_elo']

            # structs placed
            dat[prefix + '_PI'] = pd.to_numeric(dat[prefix + '_PI'], downcast='integer')
            dat[prefix + '_ac1'] = pd.to_numeric(dat[prefix + '_ac1'], downcast='integer')
            dat[prefix + '_ac2'] = pd.to_numeric(dat[prefix + '_ac2'], downcast='integer')
            dat[prefix + '_lab'] = pd.to_numeric(dat[prefix + '_lab'], downcast='integer')
            dat[prefix + '_ts'] = pd.to_numeric(dat[prefix + '_ts'], downcast='integer')
            dat[prefix + '_m'] = pd.to_numeric(dat[prefix + '_m'], downcast='integer')

            # create general structs catergory
            dat.loc[dat[prefix + '_PI'] > 0, structs_class] = 'PI'
            dat.loc[dat[prefix + '_ac1'] > 0, structs_class] = 'ac1'
            dat.loc[dat[prefix + '_ac2'] > 0, structs_class] = 'ac2'
            dat.loc[
                (dat[prefix + '_lab'] > 0) &
                (dat[prefix + '_PI'] == 0) &
                (dat[prefix + '_ac1'] == 0) &
                (dat[prefix + '_ac2'] == 0)
                , structs_class] = 'lab'

            dat.loc[
                (dat[prefix + '_ts'] > 0) &
                (dat[prefix + '_lab'] == 0) &
                (dat[prefix + '_PI'] == 0) &
                (dat[prefix + '_ac1'] == 0) &
                (dat[prefix + '_ac2'] == 0)
                , structs_class] = 'ts'

            dat.loc[
                (dat[prefix + '_ts'] == 0) &
                (dat[prefix + '_lab'] == 0) &
                (dat[prefix + '_PI'] == 0) &
                (dat[prefix + '_ac1'] == 0) &
                (dat[prefix + '_ac2'] == 0)
                , structs_class] = 'm'

            # create exact category
            dat[structs_exact] = [" " +
                                  str(m) + '-m_' +
                                  str(ts) + '-ts_' +
                                  str(lab) + '-lab_' +
                                  str(PI) + '-pi_' +
                                  str(ac1) + '-ac1_' +
                                  str(ac2) + '-ac2'
                                  for m, ts, lab, PI, ac1, ac2 in zip(
                    dat[prefix + '_m'],
                    dat[prefix + '_ts'],
                    dat[prefix + '_lab'],
                    dat[prefix + '_PI'],
                    dat[prefix + '_ac1'],
                    dat[prefix + '_ac2'])
                                  ]

            factions_pos.append(dat)

        factions_in = pd.concat(factions_pos, ignore_index=True)

        unique_starts = factions_in[[structs_exact, structs_class, over_all_score, over_all_elo]]
        unique_starts['class_counts'] = 0
        counts = unique_starts.groupby(structs_exact)['class_counts'].transform('count').to_frame()
        unique_starts = unique_starts[[structs_exact, structs_class, over_all_score, over_all_elo]]
        unique_starts = pd.concat([unique_starts, counts], axis=1)
        unique_starts = unique_starts.groupby([structs_class, structs_exact]).mean().reset_index()
        unique_starts['class_counts'] = pd.to_numeric(unique_starts['class_counts'], downcast='integer')

        # remove really low scores
        # unique_starts = unique_starts[(np.abs(stats.zscore(unique_starts[over_all_score])) > 2)]
        unique_starts = unique_starts[unique_starts[over_all_score].between(unique_starts[over_all_score].quantile(.15),
                                                                            unique_starts[over_all_score].quantile(1))]

        # get average score
        midpoint = 0
        for i in range(num_players):
            midpoint += np.mean(dat_clean['pos_' + str(i + 1) + '_score'])
        midpoint /= num_players

        # midpoint is average for all factions overall - but color ends are highest and lowest score by that faction
        fig = px.treemap(unique_starts,
                         path=[px.Constant('R1 Structures: Frequency and Overall Score for ' + faction), structs_class,
                               structs_exact], values='class_counts',
                         color=over_all_score, hover_data=[over_all_elo],
                         color_continuous_scale='RdBu',
                         color_continuous_midpoint=midpoint
                         )
        fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
        fig.update_layout(
            title_text="R1 Structures: Frequency and Impact on Score Relative to All Factions Average (" + str(
                round(midpoint, 2)) + "): " + faction)
        fig.show()


In [14]:
plot_r1_structs_remote(4, dat_4)